# TIME series model prototype

In [127]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Masking, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model, regularizers, layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

## Loading time series 

In [5]:
raw_data_prep_dir = '../raw_data/pads-parkinsons-disease-smartwatch-dataset-1.0.0/preprocessed/'

In [6]:
file_list = pd.read_csv(raw_data_prep_dir  + "file_list.csv")
file_list

,resource_type,id,study_id,condition,disease_comment,age_at_diagnosis,age,height,weight,gender,handedness,appearance_in_kinship,appearance_in_first_grade_kinship,effect_of_alcohol_on_tremor,label
0,patient,1,PADS,Healthy,-,56,56,173,78,male,right,True,True,Unknown,0
1,patient,2,PADS,Other Movement Disorders,Left-Sided resting tremor and hypokinesia with...,69,81,193,104,male,right,False,NaN,No effect,2
2,patient,3,PADS,Healthy,-,45,45,170,78,female,right,False,NaN,Unknown,0
3,patient,4,PADS,Parkinson's,IPS akinetic-rigid type,63,67,161,90,female,right,False,NaN,No effect,1
4,patient,5,PADS,Parkinson's,IPS tremordominant type,65,75,172,86,male,left,False,NaN,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,patient,465,PADS,Parkinson's,IPS mixed type,62,65,175,80,male,right,True,False,No effect,1
465,patient,466,PADS,Healthy,-,84,84,172,74,female,right,True,True,No effect,0
466,patient,467,PADS,Parkinson's,"Essential Tremor, starting IPS tremordominant ...",55,57,190,100,male,right,False,NaN,Improvement,1
467,patient,468,PADS,Parkinson's,IPS mixed type,73,76,198,118,male,right,False,NaN,No effect,1


In [42]:
n_id = len(file_list['id'])
n_id

469

In [18]:
file_list[file_list['id']==5]['label']

4    1
Name: label, dtype: int64

In [64]:
label = file_list.loc(file_list['id']==1,'label')

TypeError: _LocationIndexer.__call__() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
time_data = []
y = []
X_to_split = []
for idx, subject_idx in enumerate(file_list['id']):
    file_name = raw_data_prep_dir + f'movement/{subject_idx:03d}_ml.bin'
    time_idx_data = np.fromfile(file_name, dtype=np.float32).reshape((-1, 976))
    time_data.append(time_idx_data)
    y_idx = file_list[file_list['id']==subject_idx].label.values
    X_id = idx
    y.append(y_idx)
    X_to_split.append(X_id)

X_to_split = np.array(X_to_split)
y = np.array(y).flatten()
print(X_to_split.shape, y.shape)

(469,) (469,)


In [82]:
channels = []
for task in ["Relaxed1", "Relaxed2", "RelaxedTask1", "RelaxedTask2", "StretchHold", "HoldWeight",
                "DrinkGlas", "CrossArms", "TouchNose", "Entrainment1", "Entrainment2"]:
    for device_location in ["LeftWrist", "RightWrist"]:
        for sensor in ["Acceleration", "Rotation"]:
            for axis in ["X", "Y", "Z"]:
                channel = f"{task}_{sensor}_{device_location}_{axis}"
                channels.append(channel)
len(channels)

132

In [53]:
time_data_array = np.array(time_data)
print(time_data_array.shape)

(469, 132, 976)


In [112]:
X_train_idx, X_test_idx, y_train, y_test = train_test_split(X_to_split, y, test_size=0.20, random_state=42, stratify = y)

In [118]:
X_train = time_data_array[X_train_idx]
X_test = time_data_array[X_test_idx]
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

375 94
375 94


In [ ]:
n_obs, n_chan, n_time =  X_train.shape
print(n_obs, n_chan, n_time)
X_train_nn = X_train.reshape(n_obs, n_time, n_chan)
input_shape = X_train_nn.shape[1:]
print(input_shape)

n_obs_test, n_chan_test, n_time_test =  X_test.shape
print(n_obs_test, n_chan_test, n_time_test)
X_test_nn = X_test.reshape(n_obs_test, n_time_test, n_chan_test)
X_test_nn.shape

375 132 976
(976, 132)
94 132 976


(94, 976, 132)

In [154]:
def initialize_model_lstm(input_shape:tuple, dropout_rate: float = 0.3) -> Model:
    model = Sequential()
    model.add(Input(shape=input_shape))

    #two lstm layers to better capture
    model.add(LSTM(units=64, activation='tanh',
                   return_sequences=True,
                   dropout=dropout_rate,
                   recurrent_dropout=dropout_rate))
    model.add(LSTM(units=64, activation='tanh',
                   dropout=dropout_rate,
                   recurrent_dropout=dropout_rate))

    model.add(layers.Dropout(dropout_rate))

    #dense layer with batch normalization
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))

    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dropout(dropout_rate))

    #output layer with 3 classification
    model.add(layers.Dense(3, activation='softmax'))

    return model

In [155]:
model_lstm = initialize_model_lstm(input_shape=X_train_nn.shape[1:])

model_lstm.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=Adam(learning_rate=0.01),
    metrics=['accuracy', 'precision', 'recall'])

In [156]:
model_lstm.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 976, 64)        │        50,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 86,243 (336.89 KB)

 Trainable params: 86,179 (336.64 KB)

 Non-trainable params: 64 (256.00 B)

In [161]:
es = EarlyStopping(
    monitor="val_loss",
    patience=50,
    restore_best_weights=True,
    verbose=0
)

reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=3,
        verbose=1,
        min_lr=0
    )

history = model_lstm.fit(
    X_train_nn,
    y_train,
    validation_split = 0.2,
    epochs=64,
    batch_size = 32,
    callbacks=[es, reduce_lr],
    verbose=1
)

Epoch 1/64


InvalidArgumentError: Graph execution error:

Detected at node LogicalAnd_1 defined at (most recent call last):
  File "/Users/anna/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/Users/anna/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code

  File "/Users/anna/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/anna/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/anna/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/anna/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/Users/anna/.pyenv/versions/3.10.6/lib/python3.10/asyncio/base_events.py", line 600, in run_forever

  File "/Users/anna/.pyenv/versions/3.10.6/lib

In [141]:
input_shape

(976, 132)